In [1]:
!uv pip install sagemaker
!uv pip install boto3

Using Python 3.10.15 environment at C:\Users\bored\Music\SagemekerLearning\sagelearner
Audited 1 package in 969ms
Using Python 3.10.15 environment at C:\Users\bored\Music\SagemekerLearning\sagelearner
Audited 1 package in 9ms


In [2]:
job_name = 'iris-xgboost-2025-06-08-17-43-29'

In [ ]:
from sagemaker.estimator import Estimator
job_name = 'iris-xgboost-2025-06-08-17-43-29'
xgb_model = Estimator.attach(job_name)
prefix="iris"
bucket = 'sgmkr-learner'

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\bored\AppData\Local\sagemaker\sagemaker\config.yaml

2025-06-08 12:16:32 Starting - Preparing the instances for training
2025-06-08 12:16:32 Downloading - Downloading the training image
2025-06-08 12:16:32 Training - Training image download completed. Training in progress.
2025-06-08 12:16:32 Uploading - Uploading generated training model
2025-06-08 12:16:32 Completed - Training job completed


In [5]:
batch_ip = "s3://{}/{}/{}".format(bucket, prefix, "batch_transform")
batch_op = "s3://{}/{}/{}".format(bucket, prefix, "batch_transform")

In [6]:
!aws s3 ls {batch_ip}/ --recursive

2025-06-07 15:19:22        850 iris/batch_transform/iris_infer.csv


In [7]:
transformer = xgb_model.transformer(
    instance_count=1, instance_type="ml.m4.xlarge", output_path=batch_op
)

In [8]:
from sagemaker.transformer import Transformer

In [9]:
transformer.transform(
    data=batch_ip, data_type="S3Prefix", content_type="text/csv", split_type="Line"
)
transformer.wait()

INFO:sagemaker:Creating transform job with name: iris-xgboost-2025-06-08-17-43-29-2025-06-09-13-08-53-220


................................Arguments: serve
[2025-06-09 13:14:58 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2025-06-09 13:14:58 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2025-06-09 13:14:58 +0000] [1] [INFO] Using worker: gevent
[2025-06-09 13:14:58 +0000] [21] [INFO] Booting worker with pid: 21
[2025-06-09 13:14:58 +0000] [22] [INFO] Booting worker with pid: 22
[2025-06-09 13:14:59 +0000] [23] [INFO] Booting worker with pid: 23
[2025-06-09 13:14:59 +0000] [24] [INFO] Booting worker with pid: 24
/opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util.ssl_ (/opt/amazon/lib/python3.7/site-packages/urllib3/util/ssl_.p

In [10]:
!aws s3 ls {bucket}/{prefix}/batch_transform/ --recursive

2025-06-07 15:19:22        850 iris/batch_transform/iris_infer.csv
2025-06-09 18:45:04        200 iris/batch_transform/iris_infer.csv.out


In [11]:
!aws s3 cp s3://{bucket}/{prefix}/batch_transform/iris_infer.csv.out .

Completed 200 Bytes/200 Bytes (196 Bytes/s) with 1 file(s) remaining
download: s3://sgmkr-learner/iris/batch_transform/iris_infer.csv.out to .\iris_infer.csv.out


---

# 🚀 SageMaker Batch Transform Summary

---

### ✅ **Purpose:**

> Do inference on large datasets directly from S3 without creating real-time endpoints.

---

### ✅ **Key Advantages:**

* ✅ No always-running endpoint (cheaper for large data)
* ✅ Fully serverless → you don’t manage infrastructure
* ✅ Supports large batch jobs — process GBs/TBs of data easily
* ✅ Automatically parallelized
* ✅ Results stored directly into S3

---

### ✅ **Basic Flow:**

```python
# 1️⃣ Create transformer object
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path="s3://your-bucket/output-path"
)

# 2️⃣ Start batch transform job
transformer.transform(
    data="s3://your-bucket/input-path",
    data_type="S3Prefix",
    content_type="text/csv",
    split_type="Line"
)

# 3️⃣ Wait for completion
transformer.wait()
```

---

### ✅ **Parameters Explained:**

| Parameter                 | Meaning                                 |
| ------------------------- | --------------------------------------- |
| `instance_count`          | Number of compute instances             |
| `instance_type`           | Type of instances (e.g. `ml.m4.xlarge`) |
| `output_path`             | S3 location to store predictions        |
| `data`                    | S3 input location                       |
| `data_type="S3Prefix"`    | Read all files from S3 prefix           |
| `content_type="text/csv"` | Format of your input data               |
| `split_type="Line"`       | Split input by line (default for CSV)   |

---

### ✅ **Cost Model:**

* You only pay for the instance usage **during the job runtime**.
* Once the job finishes, compute shuts down automatically.
* Much cheaper than real-time endpoints for one-time or periodic inference.

---

### ✅ **When to use Batch Transform:**

| ✅ Good for            | ❌ Not for                       |
| --------------------- | ------------------------------- |
| Large datasets        | Low-latency real-time inference |
| Periodic scoring      | Interactive single predictions  |
| Infrequent batch jobs | Chatbots, APIs                  |

---

### ✅ **Where output goes:**

* Output is saved as CSV files into your specified `output_path` in S3.
* Files will look like:

```
s3://your-bucket/output-path/iris-xgboost-<timestamp>/output/part000.gz
```

---

### ✅ **Extra tip:**

Batch Transform *still uses your trained model artifacts* (`model.tar.gz` saved in S3) — you don’t retrain anything.

---

# 🔥 Final mental model:

```
[S3 Input Data]
       ↓
[Batch Transform Job]
       ↓
[S3 Output Predictions]
```

---



'head' is not recognized as an internal or external command,
operable program or batch file.
